In [1]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from sklearn.cross_validation import StratifiedKFold, KFold
from keras.optimizers import SGD
from keras.utils import np_utils
import numpy as np

try:
    import cPickle as pickle
except:
    import pickle

batch_size = 10
nb_classes = 2
nb_epoch = 30
data_augmentation = False

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

cvscores = []


def load_data():
    # load food100 data
    f = open("../dataset/myfood10-32.pkl", 'rb')

    d = pickle.load(f)
    data = d['trainFeatures']
    labels = d['trainLabels']
    lz = d['labels']
    #(X_train, y_train), (X_test, y_test) = cifar10.load_data()
    return data,labels,lz

def create_model():
    model = Sequential()

    model.add(Convolution2D(32, 3, 3, border_mode='same',
                            input_shape=(3, img_rows, img_cols)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(64, 3, 3, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, 3, 3))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
    
    return model
    
def train_and_evaluate_model(model, X_train, Y_train, X_test, Y_test):
    
    #model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)
    model.fit(X_train, Y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          validation_data=(X_test, Y_test),
          shuffle=True)

    scores = model.evaluate(X_test, Y_test, verbose=0)
    print (scores)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)

def train(model, X_train,Y_train):
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    # convert class vectors to binary class matrices
    X_train = X_train.astype('float32')
    X_train /= 255

    Y_train = np_utils.to_categorical(Y_train, nb_classes)
   
    model.fit(X_train, Y_train, validation_split=0.5, nb_epoch=150, batch_size=10)
    
    
if __name__ == "__main__":
    k = 2
    data, labels, lz = load_data()
    lz = np.array(lz)
    
    data /=255 
    
    print('X_train shape:', data.shape)
    print(data.shape[0], 'train samples')
    
    sfold = StratifiedKFold(y=lz, n_folds=k, shuffle=False)

    
    for train_index, test_index in sfold:
            #print ("Running Fold", i+1, "/", n_folds)
            print(train_index.shape, 'train samples')
            print(test_index.shape, 'test samples')
            train_data, test_data = data[train_index], data[test_index]
            train_label, test_label = labels[train_index], labels[test_index]
            
            model = None # Clearing the NN.
            model = create_model()
            train_and_evaluate_model(model, train_data, train_label, test_data, test_label)
    #print ("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Using Theano backend.
Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5105)


X_train shape: (2000, 3, 1024)
2000 train samples
(1000,) train samples
(1000,) test samples


Exception: Error when checking model input: expected convolution2d_input_1 to have 4 dimensions, but got array with shape (1000, 3, 1024)